In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [ ]:
#Start Spark session
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
spark=SparkSession.builder.appName("StopWords").getOrCreate()

In [ ]:
#create testing df
df = spark.createDataFrame([
    (0, "hello world"),
    (1, "greetings, earth"),
    (2, "waddup, third rock from the sun")
], ["id", "sentence"])

#create tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="raw")

#drop original column because we only need tokenized column
df=tokenizer.transform(df).drop('sentence')
df.show(truncate=False)

+---+--------------------------------------+
|id |raw                                   |
+---+--------------------------------------+
|0  |[hello, world]                        |
|1  |[greetings,, earth]                   |
|2  |[waddup,, third, rock, from, the, sun]|
+---+--------------------------------------+



In [ ]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

#instantiate remover
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

#transform df
remover.transform(df).show(truncate=False)

+---+--------------------------------------+---------------------------+
|id |raw                                   |filtered                   |
+---+--------------------------------------+---------------------------+
|0  |[hello, world]                        |[hello, world]             |
|1  |[greetings,, earth]                   |[greetings,, earth]        |
|2  |[waddup,, third, rock, from, the, sun]|[waddup,, third, rock, sun]|
+---+--------------------------------------+---------------------------+

